# Example 07 - Sig53 with YOLOv8 Classifier
This notebook showcases using the Sig53 dataset to train a YOLOv8 classification model.

---

## Import Libraries
We will import all the usual libraries, in addition to Ultralytics. You can install Ultralytics with:
```bash
pip install ultralytics
```

In [1]:
from torchsig.utils.yolo_classify import *
from torchsig.datasets.wideband_sig53 import WidebandSig53
from torchsig.transforms.transforms import Spectrogram, SpectrogramImage, Normalize, Compose, Identity
from torchsig.utils.classify_transforms import real_imag_vstacked_cwt_image
import yaml
from PIL import Image
from ultralytics import YOLO

## Prepare YOLO classificatoin trainer and Model
Datasets are generated on the fly in a way that is Ultralytics YOLO compatible. See [Ultralytics: Train Custom Data - Organize Directories](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#23-organize-directories) to learn more. 

Additionally, we create a yaml file for dataset configuration. See "classify.yaml" in Torchsig Examples.

Download desired YOLO model from [Ultralytics Models](https://docs.ultralytics.com/models/). We will use YOLOv8, specifically `yolov8n-cls.pt`

---

In [2]:
config_path = 'classify.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

overrides = config['overrides']

### Explanation of the `overrides` Dictionary

The `overrides` dictionary is used to customize the settings for the Ultralytics YOLO trainer by specifying specific values that override the default configurations. The dictionary is imported from `classify.yaml`. However, you can customize in the notebook. 

Example:

```python
overrides = {'model': 'yolov8n-cls.pt', 'epochs': 100, 'data': 'classify.yaml', 'device': 0, 'imgsz': 64}
```
A .yaml is necessary for training. Look at `classify.yaml` in the examples directory. It will contain the path to your torchsig data.

### Explanation of `image_transform` function
`YoloClassifyTrainer` allows you to pass in any transform that takes in complex I/Q and outputs an image for training. Some example transforms can be found in torchsig.utils.classify_transforms. If nothing is passed, it will default to spectrogram images. It is important to update `overrides` so that your imgsz matches output.

### Build YoloClassifyTrainer
This will instantiate the YOLO classification trainer with overrides specified above.

In [3]:
trainer = YoloClassifyTrainer(overrides=overrides, image_transform=None)

Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81038MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=classify.yaml, epochs=1, time=None, patience=100, batch=32, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=8, project=SNOWY_CLASSIFY_SPEC, name=heatmap, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

### The will begin training

In [4]:
trainer.train()

100%|██████████| 5.31M/5.31M [00:00<00:00, 49.4MB/s]


Overriding model.yaml nc=1000 with nc=53

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralyt

/usr/local/lib/python3.10/dist-packages/ultralytics/engine/trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)


args -> classify.yaml
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 8 dataloader workers
Logging results to SNOWY_CLASSIFY_SPEC/heatmap
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 829/829 [02:51<00:00,  4.83it/s]


                   all      0.671      0.999

1 epochs completed in 0.098 hours.
Optimizer stripped from SNOWY_CLASSIFY_SPEC/heatmap/weights/last.pt, 3.1MB
Optimizer stripped from SNOWY_CLASSIFY_SPEC/heatmap/weights/best.pt, 3.1MB

Validating SNOWY_CLASSIFY_SPEC/heatmap/weights/best.pt...
Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81038MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,502,773 parameters, 0 gradients, 3.3 GFLOPs


               classes   top1_acc   top5_acc: 100%|██████████| 829/829 [02:53<00:00,  4.78it/s]


                   all      0.671      0.999
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to SNOWY_CLASSIFY_SPEC/heatmap
Results saved to SNOWY_CLASSIFY_SPEC/heatmap


### Predictions / Inference
The following cells show you how to load the 'best.pt' weights from your training for prediction

In [10]:
import os
d_path = './runs/detect/'
all_subdirs = [d_path + d for d in os.listdir(d_path) if os.path.isdir(d_path + d)]
all_subdirs = [value for value in all_subdirs if value.find('train') > 0]
latest_subdir = max(all_subdirs, key=os.path.getmtime)
model_path = f'{latest_subdir}/weights/best.pt'
model = YOLO(model_path)

In [11]:
test_path = './datasets/wideband_sig53'

transform = Compose([
    Spectrogram(nperseg=512, noverlap=0, nfft=512, mode='psd'),
    Normalize(norm=np.inf, flatten=True),
    SpectrogramImage(), 
    ])
target_transform = Compose([
    ])
    
test_dataset = WidebandSig53(
            root=test_path, 
            train=True, 
            impaired=True,
            transform=transform,
            target_transform=target_transform
        )

# View data before inference
sample, _ = test_dataset[1]
img = Image.fromarray(sample)


results = model.predict(img)


0: 512x512 (no detections), 5.4ms
Speed: 0.7ms preprocess, 5.4ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 512)
